In [1]:
import numpy as np
import pandas as pd
import string
import re
import pickle
import tqdm
import seaborn as sns


import nltk 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from','like' 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come','py','gt','lt'])
char_list = ['=', '+','-','*','<','>','/',')','(','[',']',':'] 

import gensim
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook



import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])
from spacy.lang.en import English
parser = English()

# from nltk.corpus import wordnet as wn

from gensim.corpora import Dictionary
from pprint import pprint

import os 
mallet_path = 'C:/mallet-2.0.8/bin/mallet' # update this path

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

# Data pre-processing

In [2]:
def cleanhtml(raw_html):
    back_quotes = re.compile('<code>(.|\n)*?</code>')
    cleantext = re.sub(back_quotes,'', raw_html)
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', cleantext)
    rem_url = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', cleantext, flags=re.MULTILINE)
    rem_num = re.sub('[^a-zA-Z ]', ' ', rem_url)
    return rem_num
       
def tokenization(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace() | token.is_punct :
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


def get_stem(word):
    porter = PorterStemmer()
    return porter.stem(word)
    
def get_lemma(word):
    return WordNetLemmatizer().lemmatize(word)


In [3]:
def read_file(path_posts):
    posts = pd.read_csv(path_posts)
    posts.creationdate = pd.to_datetime(posts.creationdate  , format='%Y-%m-%d', errors='ignore')
    posts.AcceptedAnswerdate = pd.to_datetime(posts.AcceptedAnswerdate  , format='%Y-%m-%d', errors='ignore')
    posts.ACCanswerBody.fillna(' ',inplace=True)

    posts['Text'] = posts.title +' '+ posts.questionBody  +' '+ posts.ACCanswerBody 

    return posts

In [4]:
result = read_file('finalresult.csv')

In [5]:
def prepare_text_for_lda(text):
    text = cleanhtml(text)
    tokens = tokenization(text)
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [get_stem(token) for token in tokens]
#     tokens = [get_lemma(token) for token in tokens]
    tokens = [token.strip() for token in tokens]
    tokens = [ele for ele in tokens if all(ch not in ele for ch in char_list)] 
#     tokens = [token for token in tokens if len(token) >=4]
    return tokens

In [6]:
text_data = result['Text'].map(lambda x : prepare_text_for_lda(x))
# Print out the first rows of papers
text_data.head()
text_data.to_csv('final_result_data_processing.csv',index=False)

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(text_data))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=30) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=30)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# Remove Stop Words
texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in data_words]
# Form Bigrams
texts = [bigram_mod[doc] for doc in texts]
# Form trigrams
texts = [trigram_mod[bigram_mod[doc]] for doc in texts]

In [9]:
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
texts_out = []
for sent in texts:
    doc = nlp(" ".join(sent)) 
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
# remove stopwords once more after lemmatization
# texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    

In [10]:
import csv
with open('data_processing_bigram_trigram.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows(texts_out)

In [11]:
# Create Dictionary
id2word = corpora.Dictionary(texts_out)
# Create Corpus
texts = texts_out
# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [12]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """

    model_results = {
                 'Topics': [],
#                  'Alpha': [],
                 'model': [],
                 'Coherence': []                    
                    }
    # Alpha parameter
    alpha = list(np.arange(0.01, 1, 0.01))
    if 1==1:
        pbar = tqdm.tqdm(total=(limit-start))  
        for num_topics in range(start, limit, step):
#             for a in alpha:
                model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary,
                                                           optimize_interval=10 , 
                                                          iterations=5000,
                                                         random_seed=100
                                                        )
                model_results['model'].append(model)
                coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
                model_results['Coherence'].append(coherencemodel.get_coherence())
                model_results['Topics'].append(num_topics)
#                 model_results['Alpha'].append(a)                
                pbar.update(1)
    pbar.close()

    return model_results

In [15]:
# Can take a long time to run.
model_list = compute_coherence_values(dictionary=id2word, corpus=corpus, 
                                                        texts=texts_out, start=5, limit=40, step=1)

100%|███████████████████████████████████████████████████████████████████████████████| 35/35 [2:33:43<00:00, 263.54s/it]


In [16]:
model_list['model'][model_list['Coherence'].index(max(model_list['Coherence']))]

In [17]:
result_tuning = pd.DataFrame.from_dict(model_list)
result_tuning[result_tuning.index==result_tuning.Coherence.idxmax()]

,Topics,model,Coherence
6,11,<gensim.models.wrappers.ldamallet.LdaMallet ob...,0.453466


In [18]:
import pickle
filename = 'tuned_model_lda'
lda_model = model_list['model'][model_list['Coherence'].index(max(model_list['Coherence']))]
with open(filename, 'wb') as file:  
    pickle.dump(lda_model, file)

In [20]:
#Final paramater for topic modeling
topics = 9
alpha = 0.9
beta = 0.6

In [21]:
# # Show Topics
# LDAModel = gensim.models.LdaMulticore(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=topics, 
#                                            random_state=100,
#                                            chunksize=10000,
#                                            passes=10,
#                                            alpha=alpha,
#                                            eta=beta,
#                                           per_word_topics=True,
#                                           )
with open('tuned_model_lda','rb') as model :
    LDAModel = pickle.load(model)
pprint(LDAModel.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=LDAModel, texts=texts_out, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(5,
  [('datum', 0.06570260924237661),
   ('bit', 0.06224457717698837),
   ('send', 0.04825526563973593),
   ('byte', 0.027978623074504872),
   ('receiv', 0.0242062244577177),
   ('serial', 0.022320025149324112),
   ('start', 0.01634706067274442),
   ('packet', 0.014460861364350833),
   ('charact', 0.014303678088651368),
   ('uart', 0.011945928953159385)]),
 (0,
  [('read', 0.1004),
   ('write', 0.08266666666666667),
   ('datum', 0.05106666666666667),
   ('address', 0.044533333333333334),
   ('memori', 0.03573333333333333),
   ('ram', 0.032266666666666666),
   ('store', 0.0284),
   ('regist', 0.025333333333333333),
   ('block', 0.016666666666666666),
   ('instruct', 0.015066666666666667)]),
 (1,
  [('type', 0.061824605153782214),
   ('declar', 0.04041978387364921),
   ('function', 0.040108063175394844),
   ('vhdl', 0.025976724854530342),
   ('error', 0.02514546965918537),
   ('constant', 0.02348295926849543),
   ('variabl', 0.01818370739817124),
   ('vector', 0.014235245220282626),
  

In [22]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
                
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords ]), ignore_index=True)               
            else:
                break
        
        
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts.Text)
    Id = pd.Series(texts.id)
    sent_topics_df = pd.concat([sent_topics_df, contents,Id], axis=1)
    return(sent_topics_df)

In [23]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=LDAModel, corpus=corpus, texts=result[['Text','id']])

In [24]:
df_topic_sents_keywords.groupby('Dominant_Topic').size().reset_index(name='Frequency').to_excel('topics_distribution.xlsx')

In [25]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Sentence_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text','id']
df_dominant_topic_final = pd.merge(result ,df_dominant_topic, on='id',how='inner' )
df_dominant_topic_final.to_csv('Topic_modeling_result_HDL.csv',index=False)


In [26]:
df_freq = df_dominant_topic_final.groupby('Dominant_Topic').size().reset_index(name='Frequency')
# Calculate Percentage
df_freq['percentage'] = (df_freq['Frequency'] / 
                      df_freq['Frequency'].sum()) * 100

In [27]:
df_dominant_topic_final.Dominant_Topic = 'Topic_'+ df_dominant_topic_final.Dominant_Topic.astype(str)

In [28]:
df_dict = {}

for topic in df_dominant_topic_final.Dominant_Topic.unique():
    topic_df = df_dominant_topic_final[df_dominant_topic_final['Dominant_Topic'] == topic]
    df_dict[topic] = topic_df
    
def save_xlsx(df_dict, path):
    """
    Save a dictionary of dataframes to an excel file, with each dataframe as a seperate page
    """

    with pd.ExcelWriter(path) as writer:
        for key in df_dict:
            df_dict[key].to_excel(writer, key, index=False)

    writer.save()
save_xlsx(df_dict, 'Topic_modeling_results_per_topic.xlsx')

C:\Users\2019\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
